# Task 1

In [4]:
import pandas as pd

# Load the wine dataset
wine_df = pd.read_excel("wine.xlsx")

# Load the data dictionary
data_dictionary = pd.read_excel("DataDictionary_Wine.xlsx")

wine_df.head()


,INDEX,Purchase,Cases,STARS,FixedAcidity,VolatileAcidity,CitricAcid,ResidualSugar,Chlorides,FreeSulfurDioxide,TotalSulfurDioxide,Density,pH,Sulphates,Alcohol,LabelAppeal,AcidIndex
0,1,1,3,2,3.2,1.160,-0.98,54.2,-0.567,,268,0.99280,3.33,-0.59,9.9,0,8
1,2,1,3,3,4.5,0.160,-0.81,26.1,-0.425,15,-327,1.02792,3.38,0.7,,-1,7
2,4,1,5,3,7.1,2.640,-0.88,14.8,0.037,214,142,0.99518,3.12,0.48,22,-1,8
3,5,1,3,1,5.7,0.385,0.04,18.8,-0.425,22,115,0.99640,2.24,1.83,6.2,-1,6
4,6,1,4,2,8.0,0.330,-1.26,9.4,,-167,108,0.99457,3.12,1.77,13.7,0,9
